In [1]:
from openai import OpenAI
import requests
import json
import pandas as pd
import jieba
from typing import List, Dict,Tuple
import os
from tenacity import retry, stop_after_attempt, wait_exponential
from concurrent.futures import ThreadPoolExecutor

# deepseek
# client = OpenAI(
#     api_key="sk-2fcb2f476ae4463aae2486b99b9f85cc",
#     base_url="https://api.deepseek.com",
# )
# models="deepseek-reasoner"

# 腾讯
client = OpenAI(
    api_key="sk-oervjTZRRbLCCP7Jpl70siJcaJYtF4aDEiJSwTPfsebJv3BF",
    base_url="https://api.lkeap.cloud.tencent.com/v1",
)
models="deepseek-r1"

In [2]:
from typing import List

MAX_CHUNK = 500  # 分块大小
OVERLAP = 50     # 重叠字符数

def fast_chunk_to_list(txt_path: str) -> List[str]:
    """正确实现重叠分块"""
    if MAX_CHUNK <= OVERLAP:
        raise ValueError("MAX_CHUNK必须大于OVERLAP")
    
    chunks = []
    buffer = ""
    
    with open(txt_path, 'r', encoding='utf-8') as f:
        for line in f:
            buffer += line.strip()  # 根据需求可保留换行符
            
            # 当缓冲区足够大时持续分块
            while len(buffer) >= MAX_CHUNK:
                # 取前MAX_CHUNK字符作为当前块
                chunks.append(buffer[:MAX_CHUNK])
                
                # 保留最后OVERLAP字符作为重叠部分
                # 注意：保留的是 [MAX_CHUNK-OVERLAP : MAX_CHUNK] 这部分
                buffer = buffer[MAX_CHUNK - OVERLAP:]
    
    # 处理剩余内容
    if buffer:
        chunks.append(buffer)
    
    return chunks

In [3]:
# -------------------- API调用处理 --------------------
@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=2, max=10))
def call_tencent_api(chunk: str) -> dict:
    try:
        print(chunk)
        prompt = f"""请从以下文本提取知识图谱信息：
        1. 识别关系：源实体、目标实体、关系类型
        格式要求：
        - 关系：("源实体|目标实体|关系类型)
        2.下面这种情况就提取中文，例如：污染源（Pollution Sources），只提取污染源作为实体，不提取括号里面的英文
        3.尽可能多的提取到关系
        4.输出保证是：("源实体|目标实体|关系类型)的格式
        5.输出文本直接给出关系，关系间分行，不要其他内容,只要关系
        文本内容：{chunk.replace(" ","")}"""
        completion = client.chat.completions.create(
            model=models,
            messages=[
                {"role": "system", "content": "你是一个专业的知识图谱构建助手"},
                {"role": "user", "content": prompt}
            ]
        )
        
        result = parse_response(completion)
        print(result)
        # 添加原文验证信息
        for rel in result["relationships"]:
            rel["source_text"] = chunk
        return result
    except Exception as e:
        print("========================================")
        print(e)
        return {"entities": [], "relationships": []}

def parse_response(response) -> dict:
    relationships = []
#     full_text = (response.choices[0].message.reasoning_content or "") + "\n" + (response.choices[0].message.content or "")
    full_text = response.choices[0].message.content
    
    for line in full_text.split('\n'):
        line = line.strip()
        parts = line.split('|')
        if len(parts) >= 3:
                relationships.append({
                    "source": parts[0].strip().upper(),
                    "target": parts[1].strip().upper(),
                    "rel_type": parts[2].strip(),
                })
    return {"relationships": relationships}


In [4]:
import time
# -------------------- 主流程 --------------------
def build_knowledge_graph(txt_path: str, processed_name: str):
    # 1. 提取文本分块
    text_chunks = fast_chunk_to_list(txt_path)
    print(f"总块数：{len(text_chunks)}")
    
    # 2. 并行处理分块
    all_relationships = []
    
    with ThreadPoolExecutor(max_workers=1) as executor:
        futures = [executor.submit(call_tencent_api, chunk) for chunk in text_chunks]
        
        for future in futures:
            try:
                result = future.result()
                all_relationships.extend(result["relationships"])
            except Exception as e:
                print(f"处理失败：{str(e)}")

    # 3. 数据处理和保存
    if all_relationships:
        df_rels = pd.DataFrame(all_relationships).drop_duplicates(
            subset=['source', 'target', 'rel_type', 'source_text']
        )
        df_rels['label'] = processed_name
        # 保存包含原文验证的结果
        df_rels.to_csv(f"./result/{processed_name}_relationships.csv", index=False)
        print(f"已保存结果到：{processed_name}_relationships.csv")

# -------------------- 文件批量处理 --------------------
def process_txt_files(folder_path: str):
    """处理文件夹下所有TXT文件"""
    count=1
    for root, _, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith('.txt'):
                file_path = os.path.join(root, filename)
                # 处理文件名
                name_part = os.path.splitext(filename)[0]
                processed_name = name_part.split()[-1] if len(name_part.split()) >= 2 else name_part
                processed_name=str(count)+"_"+processed_name
                print(f"正在处理文件：{filename}，标识名称：{processed_name}")
                build_knowledge_graph(file_path, processed_name)
                count+=1

In [13]:
# 执行处理
process_txt_files("./kg_txt/")

In [34]:
# 合并结果
import glob

folder_path = './result/'

all_files = glob.glob(folder_path + "/*.csv")
dfs = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    dfs.append(df)
df = pd.concat(dfs, axis=0, ignore_index=True)

In [35]:
import re
df['rel_type']=df['rel_type']
df['开始节点类型']=df['label'].apply(lambda x:re.sub(r"[^\u4e00-\u9fff]", "", x))
df['结束节点类型']=df['label'].apply(lambda x:re.sub(r"[^\u4e00-\u9fff]", "", x))
df['开始节点']=df['source'].apply(lambda x:re.sub(r"[^\u4e00-\u9fff]", "", x))
df['结束节点']=df['target'].apply(lambda x:re.sub(r"[^\u4e00-\u9fff]", "", x))
df['关系']=df['rel_type'].apply(lambda x:re.sub(r"[^\u4e00-\u9fff]", "", x))
# 开始节点，结束节点，关系，都只提取长度小于20的
df = df[~df.apply(lambda row: any(len(row[col]) > 20 for col in ['开始节点','关系','结束节点']), axis=1)]

df=df.drop_duplicates(
    subset=['开始节点', '关系', '结束节点'],  # 指定要判断重复的列
    keep='first'  # 保留第一个出现的重复项
)

In [36]:
import re
import difflib

def fuzzy_contains(sentence, entity, threshold=0.8):
    """
    判断 sentence 中是否存在与 entity 相似度超过 threshold 的子串，
    如果存在，返回 True；否则返回 False。
    先判断精确包含，再采用滑动窗口进行模糊匹配。
    """
    # 精确匹配直接返回 True
    if entity in sentence:
        return True
    n = len(entity)
    # 尝试长度在 [n-1, n+1] 范围内的子串匹配
    for length in range(max(n-1, 1), n+2):
        for i in range(0, len(sentence) - length + 1):
            substr = sentence[i:i+length]
            ratio = difflib.SequenceMatcher(None, entity, substr).ratio()
            if ratio >= threshold:
                return True
    return False

def extract_entity_relation_context(text, entity1, entity2, threshold=0.7):
    """
    从文本中提取包含两个目标实体（entity1 和 entity2）的完整上下文。
    - 如果两实体在同一句中（模糊匹配），则返回该完整句子；
    - 否则返回包含两者的最小连续句块。
    """
    # 按中文句号、问号、感叹号分割文本，并保留句子结束符
    sentences = re.split(r'(?<=[。！？])', text)
    sentences = [s.strip() for s in sentences if s.strip()]  # 去除空白

    # 优先查找同时包含两个实体的句子（采用模糊匹配）
    for sentence in sentences:
        if fuzzy_contains(sentence, entity1, threshold) and fuzzy_contains(sentence, entity2, threshold):
            return sentence

    # 分别记录包含 entity1 与 entity2 的句子索引（模糊匹配）
    indices_entity1 = [i for i, s in enumerate(sentences) if fuzzy_contains(s, entity1, threshold)]
    indices_entity2 = [i for i, s in enumerate(sentences) if fuzzy_contains(s, entity2, threshold)]

    # 如果任一实体都未出现，则返回 None
    if not indices_entity1 or not indices_entity2:
        return text

    # 寻找最小的连续句子块，该块中两实体分别出现（模糊匹配）
    best_start, best_end = None, None
    best_length = float('inf')
    for i in indices_entity1:
        for j in indices_entity2:
            start = min(i, j)
            end = max(i, j)
            length = end - start + 1
            if length < best_length:
                best_length = length
                best_start, best_end = start, end

    context_block = ''.join(sentences[best_start:best_end+1])
    return context_block


In [37]:
df['文本'] = df.apply(
    lambda x: extract_entity_relation_context(x['source_text'], x['开始节点'], x['结束节点']),
    axis=1
)
df=df[['开始节点','开始节点类型','关系','结束节点','结束节点类型',"文本"]]
df=df.dropna()
df.to_csv("result.csv",index=False)

In [78]:
from py2neo import Graph, Node, Relationship, NodeMatcher, RelationshipMatcher
import os
import csv
import pandas as pd
def get_node_by_name(g, node_type, name):

    matcher = NodeMatcher(g)
    endnode = matcher.match(node_type, name=name).first()
    print(endnode)
    if endnode != None:
        return endnode
    else:
        return None


def get_str_by_dict(mydict):
    last = ""
    print(mydict)
    print(type(mydict))
    for key in mydict:
        last = str(key) + ":" + str(mydict[key]) + "<br>" + last
    return last


g = Graph("bolt://localhost:7687", user="neo4j", password="123456")
#从删库到跑路
cypher = 'MATCH (n) OPTIONAL MATCH (n)-[r]-() DELETE n,r'
g.run(cypher)


df=pd.read_csv("result.csv")

for index,row in df.iterrows():
   try:
        start_name = row["开始节点"]
        start_type = row["开始节点类型"].strip()
        relation = row["关系"]
        end_name = row["结束节点"]
        end_type = row["结束节点类型"].strip()
        print(row)
        start_obj = get_node_by_name(g, start_type, start_name)
        if start_obj == None:
            start_obj = Node(start_type,
                             name=start_name
                             )
            g.create(start_obj)
        # =====
        end_obj = get_node_by_name(g, end_type, end_name)
        if end_obj == None:
            end_obj = Node(end_type,
                           name=end_name
                           )
            g.create(end_obj)
        rel = Relationship(start_obj, relation, end_obj)
        g.create(rel)
   except:
        continue


开始节点                                                 制药建设项目
开始节点类型                                               制药建设项目
关系                                                       包括
结束节点                                                 化学药品制造
结束节点类型                                               制药建设项目
文本        3.9 制药建设项目 pharmaceutical constructional proje...
Name: 0, dtype: object
None
None
开始节点                                                 制药建设项目
开始节点类型                                               制药建设项目
关系                                                       包括
结束节点                                               生物生化制品制造
结束节点类型                                               制药建设项目
文本        3.9 制药建设项目 pharmaceutical constructional proje...
Name: 1, dtype: object
(_100:制药建设项目 {name: '\u5236\u836f\u5efa\u8bbe\u9879\u76ee'})
None
开始节点                                                 制药建设项目
开始节点类型                                               制药建设项目
关系                                    

(_151:制药建设项目 {name: '\u8bc4\u4ef7\u56e0\u5b50'})
None
开始节点                                                   评价因子
开始节点类型                                               制药建设项目
关系                                                       符合
结束节点                                              污染物排放总量控制
结束节点类型                                               制药建设项目
文本        符合下列基本原则之一的，应作为评价因子： a）国家或地方法规、标准中限制排放的； b）国家或...
Name: 24, dtype: object
(_151:制药建设项目 {name: '\u8bc4\u4ef7\u56e0\u5b50'})
None
开始节点                                                   评价因子
开始节点类型                                               制药建设项目
关系                                                       符合
结束节点                                       持久性有机污染物（POPS）公约
结束节点类型                                               制药建设项目
文本        符合下列基本原则之一的，应作为评价因子： a）国家或地方法规、标准中限制排放的； b）国家或...
Name: 25, dtype: object
(_151:制药建设项目 {name: '\u8bc4\u4ef7\u56e0\u5b50'})
None
开始节点                                                   评价因子
开始节点类型    

(_163:制药建设项目 {name: '\u73af\u5883\u98ce\u9669\u8bc4\u4ef7'})
None
开始节点                                               地区经济发展状况
开始节点类型                                               制药建设项目
关系                                                       影响
结束节点                                                 制药建设项目
结束节点类型                                               制药建设项目
文本        包括地区经济发展状况、居住区及人口数量、企事业单位及人员规模、相对于 制药建设项目的方位和距...
Name: 48, dtype: object
None
(_100:制药建设项目 {name: '\u5236\u836f\u5efa\u8bbe\u9879\u76ee'})
开始节点                                                    居住区
开始节点类型                                               制药建设项目
关系                                                       包含
结束节点                                                   人口数量
结束节点类型                                               制药建设项目
文本        包括地区经济发展状况、居住区及人口数量、企事业单位及人员规模、相对于 制药建设项目的方位和距...
Name: 49, dtype: object
None
None
开始节点                                                  企事业单位
开始节点类型                        

(_100:制药建设项目 {name: '\u5236\u836f\u5efa\u8bbe\u9879\u76ee'})
开始节点                                                 生态功能区划
开始节点类型                                               制药建设项目
关系                                                       约束
结束节点                                                 制药建设项目
结束节点类型                                               制药建设项目
文本        5.2 调查方法 现场踏查、相关部门走访、收集已有资料及图件（如水系图、土地利用图、环境功能...
Name: 67, dtype: object
(_166:制药建设项目 {name: '\u751f\u6001\u529f\u80fd\u533a\u5212'})
(_100:制药建设项目 {name: '\u5236\u836f\u5efa\u8bbe\u9879\u76ee'})
开始节点                                         环境影响评价技术导则土壤环境
开始节点类型                                             土壤环境（试行）
关系                                                      适用于
结束节点                                  化工、冶金、矿山采掘、农林、水利等建设项目
结束节点类型                                             土壤环境（试行）
文本        环境影响评价技术导则 土壤环境 1 适用范围 本标准规定了土壤环境影响评价的一般性原则、工作...
Name: 68, dtype: object
None
None
开始节点                                   

None
None
开始节点                                             HJ964—2018
开始节点类型                                             土壤环境（试行）
关系                                                   规定评价等级
结束节点                                                  污染影响型
结束节点类型                                             土壤环境（试行）
文本        表6 现状监测布点类型与数量 评价工作等级 占地范围内 占地范围外 7 HJ 964—201...
Name: 90, dtype: object
(_212:`土壤环境（试行）` {name: 'HJ964\u20142018'})
None
开始节点                                                     一级
开始节点类型                                             土壤环境（试行）
关系                                                  要求监测点数量
结束节点                                                 5个柱状样点
结束节点类型                                             土壤环境（试行）
文本        表6 现状监测布点类型与数量 评价工作等级 占地范围内 占地范围外 7 HJ 964—201...
Name: 91, dtype: object
None
None
开始节点                                                     一级
开始节点类型                                             土壤环境（试行）
关系                                         

...

None
开始节点                                         环境影响评价技术导则大气环境
开始节点类型                                                 大气环境
关系                                                       引用
结束节点                                                  HJ819
结束节点类型                                                 大气环境
文本        GB 3095 环境空气质量标准 HJ 2.1 建设项目环境影响评价技术导则 总纲 HJ 1...
Name: 195, dtype: object
(_345:大气环境 {name: '\u73af\u5883\u5f71\u54cd\u8bc4\u4ef7\u6280\u672f\u5bfc\u5219\u5927\u6c14\u73af\u5883'})
None
开始节点                                         环境影响评价技术导则大气环境
开始节点类型                                                 大气环境
关系                                                       引用
结束节点                                                  HJ942
结束节点类型                                                 大气环境
文本        GB 3095 环境空气质量标准 HJ 2.1 建设项目环境影响评价技术导则 总纲 HJ 1...
Name: 196, dtype: object
(_345:大气环境 {name: '\u73af\u5883\u5f71\u54cd\u8bc4\u4ef7\u6280\u672f\u5bfc\u5219\u5927\u6c14\u73af\u5883'})
None
开

None
None
开始节点                                                 GB3095
开始节点类型                                                 大气环境
关系                                                       属于
结束节点                                                 环境质量标准
结束节点类型                                                 大气环境
文本        其中环境质量标准选用 GB 3095中的环境空气质量浓度限值，如已有地方环境质量标准，应选用...
Name: 221, dtype: object
(_346:大气环境 {name: 'GB3095'})
None
开始节点                                                   估算模型
开始节点类型                                                 大气环境
关系                                                      应用于
结束节点                                               大气环境影响预测
结束节点类型                                                 大气环境
文本        3 HJ 2.2—2018 评价工作开始 项目污染源 环境空气保护 评价因子与评价 区域气象...
Name: 222, dtype: object
None
None
开始节点                                                P10%MAX
开始节点类型                                                 大气环境
关系                                                      

(_406:环评总纲 {name: '\u73af\u5883\u8981\u7d20\u73af\u5883\u5f71\u54cd\u8bc4\u4ef7\u6280\u672f\u5bfc\u5219'})
None
开始节点                                         环境要素环境影响评价技术导则
开始节点类型                                                 环评总纲
关系                                                       包含
结束节点                                             生态影响评价技术导则
结束节点类型                                                 环评总纲
文本        3.2 建设项目环境影响评价技术导则体系构成 由总纲、污染源源强核算技术指南、环境要素环境影...
Name: 256, dtype: object
(_406:环评总纲 {name: '\u73af\u5883\u8981\u7d20\u73af\u5883\u5f71\u54cd\u8bc4\u4ef7\u6280\u672f\u5bfc\u5219'})
None
开始节点                                         环境要素环境影响评价技术导则
开始节点类型                                                 环评总纲
关系                                                       包含
结束节点                                           土壤环境影响评价技术导则
结束节点类型                                                 环评总纲
文本        3.2 建设项目环境影响评价技术导则体系构成 由总纲、污染源源强核算技术指南、环境要素环境影...
Name: 257, dtype: object
(_406:

(_435:环评总纲 {name: '\u5efa\u8bbe\u9879\u76ee\u6982\u51b5'})
None
开始节点                                                 建设项目概况
开始节点类型                                                 环评总纲
关系                                                       包括
结束节点                                                   公用工程
结束节点类型                                                 环评总纲
文本        4 建设项目工程分析 4.1 建设项目概况 包括主体工程、辅助工程、公用工程、环保工程、储运...
Name: 287, dtype: object
(_435:环评总纲 {name: '\u5efa\u8bbe\u9879\u76ee\u6982\u51b5'})
None
开始节点                                                 建设项目概况
开始节点类型                                                 环评总纲
关系                                                       包括
结束节点                                                   环保工程
结束节点类型                                                 环评总纲
文本        4 建设项目工程分析 4.1 建设项目概况 包括主体工程、辅助工程、公用工程、环保工程、储运...
Name: 288, dtype: object
(_435:环评总纲 {name: '\u5efa\u8bbe\u9879\u76ee\u6982\u51b5'})
None
开始节点                                  

(_488:环评总纲 {name: '\u73af\u5883\u5f71\u54cd\u7ecf\u6d4e\u635f\u76ca\u5206\u6790'})
None
